In [59]:
model_name = "google-bert/bert-base-cased"
dataset_name = "thachhoang/cypress_test"

In [60]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain).

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [61]:
from datasets import load_dataset

dataset = load_dataset(dataset_name)
dataset["train"][1000]

{'text': "// @ts-check\n\n'use strict'\nconst path = require('path')\nconst {\n  SnapshotGenerator,\n  prettyPrintError,\n  generateBundlerMetadata,\n} = require('@tooling/v8-snapshot')\n\nconst projectBaseDir = path.join(__dirname, '../')\nconst snapshotEntryFile = require.resolve('./snapshot.js')\nconst appEntryFile = require.resolve('../app/index')\n\nconst cacheDir = path.resolve(__dirname, '../cache')\n\n;(async () => {\n  try {\n    const meta = await generateBundlerMetadata(\n      projectBaseDir,\n      snapshotEntryFile,\n      {\n        entryFile: appEntryFile,\n        nodeModulesOnly: false,\n      },\n    )\n\n    const snapshotGenerator = new SnapshotGenerator(\n      projectBaseDir,\n      snapshotEntryFile,\n      {\n        cacheDir,\n        minify: false,\n        nodeModulesOnly: false,\n        resolverMap: meta.resolverMap,\n      },\n    )\n\n    await snapshotGenerator.createScript()\n    await snapshotGenerator.makeSnapshot()\n    snapshotGenerator.installSnap

In [62]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="longest", truncation="longest_first")


tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Rename the column label to match the model's config
tokenized_datasets = tokenized_datasets.rename_column("__index_level_0__", "label")

In [63]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))

In [64]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [65]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [67]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [68]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [69]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="bert-cypress", eval_strategy="epoch", push_to_hub=True)

In [70]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_train_dataset,
    compute_metrics=compute_metrics,
)

In [71]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.041551537811756134, 'eval_accuracy': 0.004, 'eval_runtime': 16.1942, 'eval_samples_per_second': 61.75, 'eval_steps_per_second': 7.719, 'epoch': 1.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.03366152197122574, 'eval_accuracy': 0.004, 'eval_runtime': 16.1798, 'eval_samples_per_second': 61.805, 'eval_steps_per_second': 7.726, 'epoch': 2.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.03290283679962158, 'eval_accuracy': 0.004, 'eval_runtime': 19.1519, 'eval_samples_per_second': 52.214, 'eval_steps_per_second': 6.527, 'epoch': 3.0}
{'train_runtime': 226.9359, 'train_samples_per_second': 13.22, 'train_steps_per_second': 1.652, 'train_loss': 0.03491429901123047, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=0.03491429901123047, metrics={'train_runtime': 226.9359, 'train_samples_per_second': 13.22, 'train_steps_per_second': 1.652, 'total_flos': 789354427392000.0, 'train_loss': 0.03491429901123047, 'epoch': 3.0})

In [72]:
trainer.push_to_hub()
tokenizer.push_to_hub("bert-cypress")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rogery7/bert-cypress/commit/db36725d8f9b166e79c78132ec6174652ee849a0', commit_message='Upload tokenizer', commit_description='', oid='db36725d8f9b166e79c78132ec6174652ee849a0', pr_url=None, pr_revision=None, pr_num=None)